In [53]:
import geocoder
import requests
import pandas as pd
import os
import key
import csv


In [54]:
source = os.path.join("data", "djaw-locations.tsv")
all_alternate_names = os.path.join("data", "djaw-locations-alternate-names.tsv")


In [56]:
%%capture
df = pd.read_csv(source, delimiter="\t", index_col=None, header=0, on_bad_lines = 'warn')

df



,id_geonames,lat,lon,region,name_modern,name_historical
0,3435910,-34.6132,-58.3772,america,NaN,Buenos Aires
1,3441575,-34.9033,-56.1882,america,NaN,Montevideo
2,3448439,-23.5475,-46.6361,america,São Paulo,Sao Paulo
3,3448622,-29.7603,-51.1472,america,São Leopoldo,Sao Leopoldo
4,3449433,-23.9608,-46.3336,america,NaN,Santos
...,...,...,...,...,...,...
801,787461,45.0522,20.4303,se_europe,Opovo,Oppova
802,789415,45.1867,20.3190,se_europe,Knićanin,Rudolfsgnad
803,789518,45.8297,20.4653,se_europe,Kikinda,Großkikinda
804,792794,44.8975,21.4172,se_europe,Bela Crkva,Weißkirchen


In [ ]:
def alternate_names_from_id(geonameId, username):
    base_url = "http://api.geonames.org/getJSON"
    params = {
        "geonameId": geonameId,  # geonameId of the place
        "username": username,  # your geonames username
    }
    response = requests.get(base_url, params=params, timeout=20, allow_redirects=True)
    try:
        data = response.json()
    except: 
        print(geonameId)
        return None

    if response:
        altnameslist = data["alternateNames"]
        return altnameslist





In [ ]:
base_url = "http://api.geonames.org/getJSON?geonameId=3553478&username=dominikfill"
response = requests.get(base_url)
response.json()["alternateNames"]

In [ ]:
plcs = ["post", "icao", "iata", "faac", "tcid", "abbr", "link", "phon", "piny", "wkdt", "unlc", "nuts", "lauc"]

list_of_names = []
for i, (_, row) in enumerate(df.iterrows()):
    place_names = alternate_names_from_id(row["id_geonames"], key.username)
    if place_names is not None:
        for place_name in place_names:
            if "lang" in place_name.keys() and place_name["lang"] not in plcs:
                list_of_names.append([row["id_geonames"], place_name])

    

   

In [ ]:
%%capture lon
list_of_names

In [ ]:
with open(all_alternate_names, 'a', encoding='utf8', newline='') as file:
    tsv_writer = csv.writer(file, delimiter='\t', lineterminator='\n')
    tsv_writer.writerow(["name", "lang", "id_geonames"])
    for i in list_of_names:
        tsv_writer.writerow([i[1]["name"], i[1]["lang"], i[0]])